<a href="https://colab.research.google.com/github/allansuresh/myGPT/blob/main/newTransformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading the dataset

In [ ]:
#get https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt

In [ ]:
import torch

In [ ]:
with open ('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

device = 'cuda' if torch.cuda.is_available() else 'cpu'

Store all the unique values(chars) that are used in this text

In [ ]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
block_size = 256
batch_size = 64
max_iters = 3000
eval_iters = 200
eval_interval =500
num_embd = 384
num_head = 6
num_layer = 6
learning_rate = 3e-4
dropout = 0.2

Create a Mapping from Characters to Integers
(note to self: Use a more complex mapping, like sentencepiece or tiktoken once it runs successfully once)

In [ ]:
char_to_int = { ch:i for i,ch in enumerate(chars)}
int_to_char = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [char_to_int[c] for c in s]
decode = lambda c: ''.join([int_to_char[s] for s in c])

In [ ]:
import torch
data = torch.tensor(encode(text), dtype=torch.long)

In [ ]:
n = int(0.8*len(data))
training_data = data[0:n]
validation_data = data[n:]

In [ ]:
torch.manual_seed(1337)

def get_batch(split):
    data = training_data if split == 'train' else validation_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

def est_loss():
    output = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        output[split] = losses.mean()
    model.train()
    return output

xb, yb = get_batch('train')

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class Head(nn.Module):
    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(num_embd, head_size, bias=False)
        self.query = nn.Linear(num_embd, head_size, bias=False)
        self.value = nn.Linear(num_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        B, T, C = x.shape
        k = self.key(x)
        q = self.query(x)

        wei = q @ k.transpose(-2,-1) * C**-0.5
        wei = wei.masked_fill(self.tril[:T,:T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=1)
        wei = self.dropout(wei)

        v = self.value(x)
        output = wei @ v
        return output

In [ ]:
class MultiHeadAttention(nn.Module):

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(num_embd, num_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        output = torch.cat([h(x) for h in self.heads], dim=-1)
        output = self.dropout(self.proj(output))
        return output

In [ ]:
class FeedForward(nn.Module):

    def __init__(self, num_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(num_embd, 4 * num_embd),
            nn.ReLU(),
            nn.Linear(4 * num_embd, num_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
class Block(nn.Module):

    def __init__(self, num_embd, num_head):
        super().__init__()
        head_size = num_embd // num_head
        self.sa = MultiHeadAttention(num_head, head_size)
        self.ffwd = FeedForward(num_embd)
        self.ln1 = nn.LayerNorm(num_embd)
        self.ln2 = nn.LayerNorm(num_embd)

    def forward(self, x):
        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))
        return x

In [ ]:
class BigramLanguageModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, num_embd)
        self.position_embedding_table = nn.Embedding(block_size,num_embd)
        self.blocks = nn.Sequential(*[Block(num_embd, num_head=num_head) for _ in range(num_layer)])
        self.ln_f = nn.LayerNorm(num_embd)
        self.lm_head = nn.Linear(num_embd, vocab_size)

    def forward(self, index, targets=None):
        B, T = index.shape

        token_embd = self.token_embedding_table(index)
        pos_embd = self.position_embedding_table(torch.arange(T, device=device))
        embding = token_embd + pos_embd
        embding = self.blocks(embding)
        emding = self.ln_f(embding)
        logits = self.lm_head(embding)

        if targets==None:
            loss = None
            return logits, loss

        b, t, c = logits.shape
        logits = logits.view(b*t, c)
        targets = targets.view(b*t)
        loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, index, max_tokens_gen):
        for _ in range(max_tokens_gen):
            index_cond = index[:, -block_size: ]
            logits, loss = self(index_cond)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index,index_next), dim=1)
        return index

model = BigramLanguageModel()
model = model.to(device)
logits, loss =  model(xb,yb)
print(logits.shape)
print(loss)


#print(decode(model.generate(index = torch.zeros((1,1), dtype=torch.long, device=device),max_tokens_gen=100000)[0].tolist()))

torch.Size([16384, 65])
tensor(4.5147, grad_fn=<NllLossBackward0>)


Add PyTorch optimizer (print losses for every eval_interval)

In [ ]:
optimizer = torch.optim.AdamW(model.parameters(), learning_rate)

In [ ]:
for iter in range(max_iters):

    if iter % eval_interval == 0:
        losses = est_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, eval loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

In [ ]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
print(decode(model.generate(context,max_tokens_gen=500)[0].tolist()))